In [1]:
# importing utilities
import os
import sys
from datetime import datetime

# importing visualization tools
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
%matplotlib inline
sns.set_style('darkgrid')
from IPython.display import Image, display

# importing data science libraries
import pandas as pd

In [2]:
!nvidia-smi

Sat Apr 21 02:43:23 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 390.48                 Driver Version: 390.48                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 1080    Off  | 00000000:26:00.0  On |                  N/A |
| 21%   48C    P0    46W / 200W |    838MiB /  8118MiB |      2%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
fraud_dataset = pd.read_csv('./data/PS_20174392719_1491204439457_log.csv')
print("There are ", len(fraud_dataset), " samples")

columns=['dest_type', "dest_id"]
dest_split = fraud_dataset.nameDest.str.split('(\d+)').tolist()
# print(dest_split)
dest_split = pd.DataFrame([x[:2] for x in dest_split], columns=columns)
dest_split[columns[1]] = pd.to_numeric(dest_split[columns[1]])
# encode categorical attributes into a binary one-hot encoded representation
nameDest_categ_transformed = pd.concat([pd.get_dummies(dest_split[columns[0]]), dest_split[columns[1]]], axis = 1)
nameDest_categ_transformed = nameDest_categ_transformed.rename(columns = {"C":"C_dest", "M":"M_dest"})

columns=['orig_type', "orig_id"]
orig_split = fraud_dataset.nameOrig.str.split('(\d+)').tolist()
orig_split = pd.DataFrame([x[:2] for x in orig_split], columns=columns)
# print(orig_split.head(10))
orig_split[columns[1]] = pd.to_numeric(orig_split[columns[1]])
# encode categorical attributes into a binary one-hot encoded representation
orig_type = pd.get_dummies(orig_split[columns[0]])
# print(type(orig_split))
M_orig = orig_type.copy(deep=True)
orig_type = orig_type.rename(columns = {"C":"C_orig"})
M_orig = M_orig.rename(columns = {"C":"M_orig"})
M_orig[:] = 0
nameOrig_categ_transformed = pd.concat([orig_type, M_orig, orig_split[columns[1]]], axis = 1)

type_categ_transformed = pd.get_dummies(fraud_dataset["type"])

transformed_datatset = pd.concat([fraud_dataset, type_categ_transformed, nameOrig_categ_transformed, nameDest_categ_transformed], axis = 1)

drop_labels = ["type", "nameOrig", "nameDest", "isFraud", "isFlaggedFraud"]
for label in drop_labels:
    transformed_datatset.pop(label)
    
print(transformed_datatset.shape)
print(transformed_datatset.head(10))

There are  6362620  samples
(6362620, 11)
(6362620, 22)
   step    amount  oldbalanceOrg  newbalanceOrig  oldbalanceDest  \
0     1   9839.64      170136.00       160296.36             0.0   
1     1   1864.28       21249.00        19384.72             0.0   
2     1    181.00         181.00            0.00             0.0   
3     1    181.00         181.00            0.00         21182.0   
4     1  11668.14       41554.00        29885.86             0.0   
5     1   7817.71       53860.00        46042.29             0.0   
6     1   7107.77      183195.00       176087.23             0.0   
7     1   7861.64      176087.23       168225.59             0.0   
8     1   4024.36        2671.00            0.00             0.0   
9     1   5337.77       41720.00        36382.23         41898.0   

   newbalanceDest  CASH_IN  CASH_OUT  DEBIT  PAYMENT  TRANSFER  C_orig  \
0            0.00        0         0      0        1         0       1   
1            0.00        0         0      0    